In [45]:
from google_play_scraper import app, Sort, reviews_all
from app_store_scraper import AppStore
import pandas as pd
import numpy as np
import json, os, uuid
import csv

In [47]:
with open('app_results.csv', 'r') as file:
    reader = csv.reader(file)
    header = next(reader, None)
    for row in reader:
        if len(row) < 3:
            continue  # Skip rows with insufficient elements

        # Access columns by their index
        title = row[0] if len(row) > 0 else None
        app_id = row[1] if len(row) > 1 else None
        app_store_id = row[2] if len(row) > 2 else None
        lang=row[4] if len(row) > 4 else None
        link=row[3] if len(row) > 3 else None

        g_reviews = reviews_all(
            app_id,
            sleep_milliseconds=0, # defaults to 0
            lang='en', # defaults to 'en'
            country='us', # defaults to 'us'
            sort=Sort.NEWEST, # defaults to Sort.MOST_RELEVANT
            )
        #a_reviews = AppStore(lang,link, str(app_store_id))
        if app_store_id != '':
            a_reviews = AppStore(lang, link, int(app_store_id))
        else:
        # Handle the case when app_store_id is an empty string
        # You can choose to skip or handle this situation based on your requirements
            a_reviews = None  # Replace with the desired behavior
        if a_reviews is not None:
            a_reviews.review()
            if hasattr(a_reviews, 'reviews') and isinstance(a_reviews.reviews, list):
                a_df = pd.DataFrame(a_reviews.reviews, columns=['review'])
                a_df2 = a_df.join(pd.DataFrame(a_df.pop('review').tolist()))
            else:
                # Handle the case when a_reviews.reviews is not present or not in the expected format
                a_df2 = pd.DataFrame(columns=['review'])  # Create an empty DataFrame or replace with desired behavior
        else:
            # Handle the case when a_reviews is None
            a_df2 = pd.DataFrame(columns=['review'])  # Create an empty DataFrame or replace with desired behavior
    
        g_df = pd.DataFrame(np.array(g_reviews),columns=['review'])
        g_df2 = g_df.join(pd.DataFrame(g_df.pop('review').tolist()))
        
        columns_to_drop = ['userImage', 'reviewCreatedVersion']
        columns_existing = [col for col in columns_to_drop if col in g_df2.columns]
        g_df2.drop(columns=columns_existing, inplace=True)

        #g_df2.drop(columns={'userImage', 'reviewCreatedVersion'},inplace = True)
        g_df2.rename(columns= {'score': 'rating','userName': 'user_name', 'reviewId': 'review_id', 'content': 'review_description', 'at': 'review_date', 'replyContent': 'developer_response', 'repliedAt': 'developer_response_date', 'thumbsUpCount': 'thumbs_up'},inplace = True)
        if not g_df2.empty and 'review_title' not in g_df2.columns:
            g_df2.insert(loc=3, column='review_title', value=None)
            
        g_df2.insert(loc=0, column='source', value='Google Play')
        #g_df2.insert(loc=3, column='review_title', value=None)
        g_df2['laguage_code'] = 'en'
        g_df2['country_code'] = 'us'
        
        #a_df = pd.DataFrame(np.array(a_reviews.reviews),columns=['review'])
        #a_df2 = a_df.join(pd.DataFrame(a_df.pop('review').tolist()))
        if 'isEdited' in a_df2.columns:
            a_df2.drop(columns=['isEdited'], inplace=True)

        #a_df2.drop(columns={'isEdited'},inplace = True)
        a_df2.insert(loc=0, column='source', value='App Store')
        a_df2['developer_response_date'] = None
        a_df2['thumbs_up'] = None
        a_df2['laguage_code'] = 'en'
        a_df2['country_code'] = 'us'
        a_df2.insert(loc=1, column='review_id', value=[uuid.uuid4() for _ in range(len(a_df2.index))])
        a_df2.rename(columns= {'review': 'review_description','userName': 'user_name', 'date': 'review_date','title': 'review_title', 'developerResponse': 'developer_response'},inplace = True)
        a_df2 = a_df2.where(pd.notnull(a_df2), None)
        result = pd.concat([result, g_df2, a_df2])
result


2023-06-05 18:56:18,795 [INFO] Base - Initialised: AppStore('fr', 'outcomes4me-cancer-care', 1404382419)
2023-06-05 18:56:18,796 [INFO] Base - Ready to fetch reviews from: https://apps.apple.com/fr/app/outcomes4me-cancer-care/id1404382419
2023-06-05 18:56:19,510 [INFO] Base - [id:1404382419] Fetched 1 reviews (1 fetched in total)
2023-06-05 18:56:21,919 [INFO] Base - Initialised: AppStore('fr', 'zenonco-cancer-care', 1625128857)
2023-06-05 18:56:21,919 [INFO] Base - Ready to fetch reviews from: https://apps.apple.com/fr/app/zenonco-cancer-care/id1625128857
2023-06-05 18:56:41,538 [ERROR] Base - Something went wrong: HTTPSConnectionPool(host='amp-api.apps.apple.com', port=443): Max retries exceeded with url: /v1/catalog/fr/apps/1625128857/reviews?l=en-GB&offset=0&limit=20&platform=web&additionalPlatforms=appletv%2Cipad%2Ciphone%2Cmac (Caused by ResponseError('too many 404 error responses'))
2023-06-05 18:56:41,541 [INFO] Base - [id:1625128857] Fetched 0 reviews (0 fetched in total)
2023

2023-06-05 19:04:09,109 [ERROR] Base - Something went wrong: HTTPSConnectionPool(host='amp-api.apps.apple.com', port=443): Max retries exceeded with url: /v1/catalog/us/apps/1531317081/reviews?l=en-GB&offset=0&limit=20&platform=web&additionalPlatforms=appletv%2Cipad%2Ciphone%2Cmac (Caused by ResponseError('too many 429 error responses'))
2023-06-05 19:04:09,113 [INFO] Base - [id:1531317081] Fetched 0 reviews (0 fetched in total)
2023-06-05 19:04:10,723 [INFO] Base - Initialised: AppStore('us', 'asco-guidelines', 1238827183)
2023-06-05 19:04:10,725 [INFO] Base - Ready to fetch reviews from: https://apps.apple.com/us/app/asco-guidelines/id1238827183
2023-06-05 19:04:29,938 [ERROR] Base - Something went wrong: HTTPSConnectionPool(host='amp-api.apps.apple.com', port=443): Max retries exceeded with url: /v1/catalog/us/apps/1238827183/reviews?l=en-GB&offset=0&limit=20&platform=web&additionalPlatforms=appletv%2Cipad%2Ciphone%2Cmac (Caused by ResponseError('too many 429 error responses'))
2023

2023-06-05 19:08:27,703 [ERROR] Base - Something went wrong: HTTPSConnectionPool(host='amp-api.apps.apple.com', port=443): Max retries exceeded with url: /v1/catalog/us/apps/6443982515/reviews?l=en-GB&offset=0&limit=20&platform=web&additionalPlatforms=appletv%2Cipad%2Ciphone%2Cmac (Caused by ResponseError('too many 429 error responses'))
2023-06-05 19:08:27,707 [INFO] Base - [id:6443982515] Fetched 0 reviews (0 fetched in total)
2023-06-05 19:08:29,171 [INFO] Base - Initialised: AppStore('gb', 'brian-the-brain-tumour-app', 1528956116)
2023-06-05 19:08:29,174 [INFO] Base - Ready to fetch reviews from: https://apps.apple.com/gb/app/brian-the-brain-tumour-app/id1528956116
2023-06-05 19:08:29,578 [INFO] Base - [id:1528956116] Fetched 5 reviews (5 fetched in total)
2023-06-05 19:08:30,995 [INFO] Base - Initialised: AppStore('us', 'owise-prostate-cancer-support', 1536593910)
2023-06-05 19:08:30,996 [INFO] Base - Ready to fetch reviews from: https://apps.apple.com/us/app/owise-prostate-cancer

,source,laguage_code,country_code,review_id,0,developer_response_date,thumbs_up,user_name,review_description,review_title,rating,review_date,developer_response,appVersion
0,App Store,en,us,f393ff46-6365-46dc-adba-22770fbce88e,Downloaded app. Allowed necessary notificatio...,NaT,None,NaN,NaN,NaN,NaN,NaT,NaN,NaN
1,App Store,en,us,4fddbcfa-c8c6-426a-a00b-002a51205a70,"Downloaded the app, answered the first questio...",NaT,None,NaN,NaN,NaN,NaN,NaT,NaN,NaN
0,Google Play,en,us,99c32890-0ee6-43e3-af59-9dbc492ac802,NaN,2023-04-28 20:21:21,0,Tra Cy,Auto install that I didn't want. So stop FORCI...,NaN,1.0,2023-03-23 16:19:53,Thanks for your feedback. We are sorry you had...,None
1,Google Play,en,us,70cd829a-9f25-41df-8c34-94ffac5158f2,NaN,NaT,15,Grace Shank,I think this app is a fantastic idea. I am loo...,NaN,4.0,2023-03-15 16:11:16,None,5.3.7
2,Google Play,en,us,72dba08f-e328-4c49-87bc-d4f00f1eb1f7,NaN,NaT,6,Sue Manah Buteau,I'm so surprised at the information that I've ...,NaN,5.0,2023-03-09 18:40:00,None,5.3.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,Google Play,en,us,59d2d96d-9985-4b4d-9f5e-9dd7afa71b0b,NaN,NaT,3,Manohar N,This is one of the best app to track all the c...,NaN,5.0,2021-09-24 12:03:49,None,1.2.9 Beta
77,Google Play,en,us,a075f427-e43a-41c6-856e-9f458b38c510,NaN,NaT,4,Kumar G,Best app for the Cancer patients,NaN,5.0,2021-09-16 11:22:19,None,1.2.7 Beta
78,Google Play,en,us,ad8ff48a-796d-4b29-b7ca-496872f41a55,NaN,NaT,3,Satya Prakash,Great addition to already amazing services off...,NaN,5.0,2021-09-01 13:50:54,None,None
0,Google Play,en,us,f01a98cb-78d3-4bab-aaa4-612aeb47fcdd,NaN,NaT,0,A Google user,I found this app day 1 of diagnoses and it ans...,NaN,5.0,2019-07-18 00:01:55,None,2019.4.23
